In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Creating Legal Entities and Instrument Relationships

This notebook demonstrates how you can create legal entities with multiple identifiers from different sources and utilise legal entity-instruments relationships to filter a universe of instruments based on the properties attached to the legal entities.

Attributes
----------
legal entities
relationships
"""

toggle_code("Toggle Docstring")

# Creating Legal Entities and Instrument Relationships
This notebook demonstrates that with LUSID, you can collate information from various sources onto one legal entity and use that collection of data to inform your decisions for creating an investable universe of instruments. We will:

 - Add multiple identifiers from different sources to one legal entity (LE) by merging multiple legal entity data sources.
 - Enrich legal entities with 2 different sources of Diversity, Equity and Inclusion data
 - Link instruments to their legal entities using legal entity-instrument relationships
 - Use instrument identifers to query properties on legal entities via the LE-instrument relationships created
 - Create a universe of instruments filtered by the DEI data stored as properties on the instruments' legal entities

In [2]:
import os
import pandas as pd
import datetime
import json
import pytz
from datetime import datetime
from IPython.core.display import HTML

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm
import logging
logging.basicConfig(level=logging.INFO)

from lusidtools import cocoon as lpt
import fbnsdkutilities.utilities as utils

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path=os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path=os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = utils.ApiClientFactory(
    lu,
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status=pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

In [3]:
relationship_definitions_api = api_factory.build(lu.RelationshipDefinitionsApi)
relationships_api = api_factory.build(lu.RelationshipsApi)
property_definitions_api = api_factory.build(lu.PropertyDefinitionsApi)
legal_entities_api = api_factory.build(lu.LegalEntitiesApi)
instruments_api = api_factory.build(lu.InstrumentsApi)
search_api = api_factory.build(lu.SearchApi)

In [4]:
id_scope = "default"
dei_id_scope = 'deiId'
inst_scope = "esgDemoInsts"
dei_scopeA = "deiDataSourceA"
dei_scopeB = "deiDataSourceB"

### Load a DataFrame of Legal Entity Identifiers (LEIs) for UK equities.

In [5]:
df_legal_entities=pd.read_csv("data/legal_entities.csv")
df_legal_entities.head()

A legal entity is represented as a collection of properties. Identifier properties are defined separately and used to find the legal entity while other properties are used to attach more information to the legal entity.

## Create identifier property
First we must create the identifier property for the Legal Entity. Set domain to "LegalEntity" and constraint style to "Identifier" so lusid knows it is an identifier property for a legal entity.


In [6]:
# create identifier property definition

def create_property_identifier(code, display_name):
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="LegalEntity",
                scope=id_scope,
                code=code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code="string"),
                life_time="Perpetual",
                constraint_style="Identifier"
            )
        )
    except:
        print("Property already exists")

This identifier property will be the standard Legal Entity Identifer (LEI) from the DataFrame we loaded.

In [7]:
create_property_identifier("LEI", "Legal Entity Identifier")

## Create Legal Entities

In [8]:
def create_legal_entity(identifier_property, LEI, LE_name):

    legal_entity_request=lm.UpsertLegalEntityRequest(
        identifiers={f"LegalEntity/{id_scope}/{identifier_property}":lm.PerpetualProperty(
                        key=f"LegalEntity/{id_scope}/{identifier_property}",
                        value=lm.PropertyValue(label_value=LEI)
                        ),
                    },

        display_name=LE_name,
        description=LE_name,
        counterparty_risk_information=None,
        )

    legal_entities_api.upsert_legal_entity(legal_entity_request)

In [9]:
legal_entities = []

for i in enumerate(df_legal_entities["LEI"]):
    identifier_property="LEI"
    LEI=df_legal_entities["LEI"][i[0]]
    LE_name=df_legal_entities["entityName"][i[0]]
    
    create_legal_entity(identifier_property, LEI, LE_name)
    legal_entities.append(LEI)
    
print(len(legal_entities), "legal entities created")

#### Add non-standard identidiers from another source

The next step is to add another identifier to the legal entities we just created as some data sources may have custom symbols in addition to the standard LEI. Here, we are adding a unique ID Number from another data source. This is also one of the datasets we will use to add diversity data to our properties later on in this notebook.

In [10]:
# Load a DataFrame of Diversity, Equity and Inclusion (DEI) data
df_dei_sourceA=pd.read_csv("data/dei_data_sourceA.csv")
df_dei_sourceA.head()

We need to create another identifier property for this custom identifier.

In [11]:
# create identifier property for   unique IDs

def create_property_identifier(scope, code, display_name):
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="LegalEntity",
                scope=scope,
                code=code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code="string"),
                life_time="Perpetual",
                constraint_style="Identifier"
            )
        )
    except:
        print("Property already exists")

In [12]:
create_property_identifier(dei_id_scope, "Custom_ID", "Custom ID")

The standard LEI from this secondary data source is uploaded using the first identifer property we create so LUSID knows which legal entity is being modified.

In [13]:
# update legal entities with custom identfiers
def update_legal_entity(identifier_property, LEI, internal_ID_property, internal_ID, LE_name):

    legal_entity_request=lm.UpsertLegalEntityRequest(
        identifiers={
            f"LegalEntity/{id_scope}/{identifier_property}": lm.PerpetualProperty(
            key=f"LegalEntity/{id_scope}/{identifier_property}",
            value=lm.PropertyValue(
                label_value=LEI
                )
            ),
            f"LegalEntity/{dei_id_scope}/{internal_ID_property}": lm.PerpetualProperty(
            key=f"LegalEntity/{dei_id_scope}/{internal_ID_property}",
            value=lm.PropertyValue(
                label_value=internal_ID
                )
            ),   
        },
        display_name=LE_name,
        description=LE_name,
        counterparty_risk_information=None,
    )

    legal_entities_api.upsert_legal_entity(legal_entity_request)

In [14]:
dei_sourceA_LEs = []

for i in df_dei_sourceA["Legal Entity Identifier"].index:
    identifier_property="LEI"
    internal_ID_property="Custom_ID"
    dei_sourceA_LEI=df_dei_sourceA["Legal Entity Identifier"][i]
    dei_sourceA_LE_name=df_dei_sourceA["Name"][i]
    internal_ID=str(df_dei_sourceA["Custom Legal Entity Identifier "][i])
    
    LEI = update_legal_entity(identifier_property, dei_sourceA_LEI, internal_ID_property, internal_ID, dei_sourceA_LE_name)
    dei_sourceA_LEs.append(LEI)
    
print(len(dei_sourceA_LEs), "legal entities updated")

In [15]:
# create dict for renaming df columns

column_rename_mapping = {"display_name":"displayName",
                         "lusid_legal_entity_id":"lusidLegalEntityId",
                         f"identifiers.LegalEntity/{dei_id_scope}/Custom_ID.value.label_value":"customLegalEntityIdentifier",
                         f"identifiers.LegalEntity/{id_scope}/LEI.value.label_value":"legalEntityIdentifier",
                         f"identifiers.LegalEntity/{id_scope}/LEI.key":"legalEntityIdentifierKey",
                         f"identifiers.LegalEntity/{id_scope}/LEI.value.label_value":"legalEntityIdentifier",
                         f"identifiers.LegalEntity/{dei_id_scope}/Custom_ID.key":"customLegalEntityIdentifierKey",
                         f"identifiers.LegalEntity/{dei_id_scope}/Custom_ID.value.label_value":"customLegalEntityIdentifier",
                         f"properties.LegalEntity/{dei_scopeA}/GenderScore.key":"genderScoreKey",
                         f"properties.LegalEntity/{dei_scopeA}/GenderScore.value.metric_value.value":"genderScore",
                         f"properties.LegalEntity/{dei_scopeA}/TotalDEIScore.key":"totalDEIScoreKey",
                         f"properties.LegalEntity/{dei_scopeA}/TotalDEIScore.value.metric_value.value":"totalDEIScore",
                         f"properties.LegalEntity/{dei_scopeB}/RaceEthnicityScore.key":"raceEthnicityScoreKey",
                         f"properties.LegalEntity/{dei_scopeB}/RaceEthnicityScore.value.metric_value.value":"raceEthnicityScore",
                         "values.0.related_entity.display_name":"displayName",
                         "values.0.related_entity.identifiers.1.identifier_value":"instrumentIsin"
                        }


In [16]:
# list legal entities with both identifiers
LE_list = legal_entities_api.list_legal_entities(
    id_type_scope=dei_id_scope,
    id_type_code="Custom_ID",
)

In [17]:
df_LE_list = lusid_response_to_data_frame(LE_list, 
                                          rename_properties=True, 
                                          column_name_mapping=column_rename_mapping)
df_LE_list[["displayName", 
            "description",
            "lusidLegalEntityId", 
            "customLegalEntityIdentifier",
            "legalEntityIdentifier"]].dropna().head()

## Enhance Legal Entity Information

Now we can create other properties to enhance the legal entity infomation. In this case, we will be updating the legal entities with Diversity, Equity and Inclusion (DEI) data from source A.

In [18]:
# create properties for legal entities

def create_property(scope, dtype, code, display_name):
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="LegalEntity",
                scope=scope,
                code=code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code=dtype),
                life_time="Perpetual",
            )
        )
    except:
        print("Property already exists")

In [19]:
create_property(dei_scopeA, "number", "TotalDEIScore", "Total DEI Score")
create_property(dei_scopeA, "number", "GenderScore", "Gender Score")

In [20]:
# adding properties for DEI data to existing legal entities

def enhance_legal_entity_sourceA(identifier_property, internal_ID_property, LEI, internal_ID, LE_name, TotalDEIScore, GenderScore):

    legal_entity_request= lm.UpsertLegalEntityRequest(
        identifiers={
            f"LegalEntity/{id_scope}/{identifier_property}": lm.PerpetualProperty(
            key=f"LegalEntity/{id_scope}/{identifier_property}",
            value=lm.PropertyValue(
                label_value=LEI
                )
            ), 
        },
        properties={
            f"LegalEntity/{dei_scopeA}/TotalDEIScore":
            lm.PerpetualProperty(
                key=f"LegalEntity/{dei_scopeA}/TotalDEIScore",
                value=lm.PropertyValue(
                    metric_value=lm.MetricValue(TotalDEIScore)
                )
            ),
            f"LegalEntity/{dei_scopeA}/GenderScore":
            lm.PerpetualProperty(
                key=f"LegalEntity/{dei_scopeA}/GenderScore",
                value=lm.PropertyValue(
                    metric_value=lm.MetricValue(GenderScore)
                )
            ),
        },
        display_name=LE_name,
        description=LE_name,
        counterparty_risk_information=None,
    )

    legal_entities_api.upsert_legal_entity(legal_entity_request)

In [21]:
enhanced_LEs = []

for i in df_dei_sourceA["Legal Entity Identifier"].index:
    identifier_property="LEI"
    internal_ID_property="Custom_ID"
    dei_source_LEI=df_dei_sourceA["Legal Entity Identifier"][i]
    dei_source_LE_name=df_dei_sourceA["Name"][i]
    internal_ID=str(df_dei_sourceA["Custom Legal Entity Identifier "][i])
    TotalDEIScore=df_dei_sourceA["Total DEI Score"][i]
    GenderScore=df_dei_sourceA["Total Gender Score"][i]
    
    enhance_LE = enhance_legal_entity_sourceA(identifier_property, internal_ID_property, dei_source_LEI, internal_ID, dei_source_LE_name, TotalDEIScore, GenderScore)
    enhanced_LEs.append(LEI)
    
print(len(enhanced_LEs), "legal entitites updated")

#### Uploading DEI data from a second source.

In [22]:
df_dei_sourceB = pd.read_csv("data/dei_data_sourceB.csv")
df_dei_sourceB.head()

In [23]:
create_property(dei_scopeB, "number", "RaceEthnicityScore", "Race/Ethnicity Score")

In [24]:
# adding properties for DEI data to existing legal entities

def enhance_legal_entity_sourceB(identifier_property, LEI, LE_name, RaceEthnicityScore):

    legal_entity_request=lm.UpsertLegalEntityRequest(
        identifiers={
            f"LegalEntity/{id_scope}/{identifier_property}": 
            lm.PerpetualProperty(
                key=f"LegalEntity/{id_scope}/{identifier_property}",
                value=lm.PropertyValue(
                    label_value=LEI
                )
            ), 
        },
        properties={
            f"LegalEntity/{dei_scopeB}/RaceEthnicityScore":
            lm.PerpetualProperty(
                key=f"LegalEntity/{dei_scopeB}/RaceEthnicityScore",
                value=lm.PropertyValue(
                    metric_value=lm.MetricValue(RaceEthnicityScore)
                )
            ),
        },
        display_name=LE_name,
        description=LE_name,
        counterparty_risk_information=None,
    )

    legal_entities_api.upsert_legal_entity(legal_entity_request)

In [25]:
enhanced_LEs = []

for i in df_dei_sourceB["Legal Entity Identifier"].index:
    identifier_property="LEI"
    dei_sourceB_LEI=df_dei_sourceB["Legal Entity Identifier"][i]
    dei_sourceB_LE_name=df_dei_sourceB["Name"][i]
    RaceEthnicityScore=df_dei_sourceB["Total Race/Ethnicity Score"][i]
    
    enhance_LE = enhance_legal_entity_sourceB(identifier_property, dei_sourceB_LEI, dei_sourceB_LE_name, RaceEthnicityScore)
    enhanced_LEs.append(LEI)
    
print(len(enhanced_LEs), "legal entitites updated")

## Create Instrument-Legal Entity Relationships

#### Loading DataFrame with ISINs for UK equities and corresponding LEIs
One legal entity can have multiple instruments linked to it in LUSID via instrument identifers. However, in this sample data set, each LEI is linked to only one instrument using the instrument ISIN.

In [26]:
df_inst_ISINs = pd.read_csv("data/Instrument_ISINs.csv")
df_inst_ISINs.head()

#### Upserting instruments 
Now we need to create instruments with these ISINs in LUSID to be able to create a relationship with their legal entity.

In [27]:
# instrument field mapping

inst_mapping = {
    "instruments": {
        "identifier_mapping": {"ClientInternal": "ClientInternal", "Isin": "ISIN"},
        "required": {
                        "name": "Name", 
                        "definition.dom_ccy":"$GBP",
                        "definition.asset_class":"$Equities",
                        "definition.instrument_type": "$Equity",
                    },
    },
}

In [28]:
df_inst_unique=df_inst_ISINs[["ISIN",
                            "ClientInternal",
                            "Name",
                             ]].drop_duplicates()
df_inst_unique.head()

In [29]:
result=lpt.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=inst_scope,
    data_frame=df_inst_unique,
    mapping_required=inst_mapping["instruments"]["required"],
    mapping_optional={},
    file_type="instrument",
    identifier_mapping=inst_mapping["instruments"]["identifier_mapping"],
    instrument_scope=inst_scope
)

succ, failed, errors=format_instruments_response(result)
print(f"number of successful upserts: {len(succ)}")
print(f"number of failed upserts    : {len(failed)}")
print(f"number of errors            : {len(errors)}")

### Create relationship between legal entity and instrument

LUSID only accepts unique identifiers when creating relationships. Since ISIN is not a unique identifier, we need a resolver to return the instrument LUID for a given ISIN. Note that this resolver will only work if there is only one instrument with the ISIN in the defined scope. 

In [30]:
# creating resolver to get unique ID for instrument 

def resolver(scope, inst_type, inst_id):
    
    # get the list of LUIDS associated with the instrument
    def no_of_insts(scope, inst_type, inst_id):
        insts_search=search_api.instruments_search(
                                            scope=scope, 
                                            mastered_only=True,
                                            instrument_search_property=[
                                            lm.InstrumentSearchProperty(
                                                key=f"Instrument/default/{inst_type}",
                                                value=inst_id,
                                            )
        ])
        return(insts_search)
    insts_search = no_of_insts(scope, inst_type, inst_id)
    no_insts = len(insts_search[0].mastered_instruments)
    
    if no_insts == 1:
        InstrumentId = insts_search[0].mastered_instruments[0].identifiers["LusidInstrumentId"].value
        
    else:
        InstrumentId=inst_id
        
    return(InstrumentId)

We can test this resolver to show that it brings back the LUID where only one instrument with the given ISIN is found in the instrument scope.

In [31]:
# test for resolver 
resolver(inst_scope, "Isin", "GB0002634946")

In [32]:
# create Legal Entity-Instrument relationship in the default scope
def create_instrument_relationship(relationship_scope, relationship_code, lei_scope, lei_code, lei, inst_scope, inst_type, inst_id):
    relationships_api.create_relationship(
        
        # the scope/code of the RelationDefinition to be created
        scope=relationship_scope,
        code=relationship_code,
        
        create_relationship_request=lm.CreateRelationshipRequest(
            target_entity_id={
                # the fields the uniquely identify the source entity
                "idTypeScope": lei_scope,
                "idTypeCode": lei_code,
                "code": lei
            }, 
            source_entity_id={
                # the fields the uniquely identify the target entity
                "scope": inst_scope,
                "identifierScope": "default",
                "identifierType": "LusidInstrumentId",
                "identifierValue": resolver(inst_scope, inst_type, inst_id)
            },
            effective_from = "2022-01-01T00:00:00+00:00"
            ),
    )

In [33]:
for i in range(len(df_inst_ISINs["ISIN"])):
    create_instrument_relationship(id_scope,
                                   "issuer",
                                   id_scope,
                                   "LEI", 
                                   df_inst_ISINs["LEI"][i], 
                                   inst_scope, 
                                   "Isin", 
                                   df_inst_ISINs["ISIN"][i])
    
print(f"Created {len(df_inst_ISINs['ISIN'])} relationships between legal entities and instruments.")

### Return legal entitiy information for a given ISIN using relationships
Now that we have relationships connecting our instruments to their legal entities, we can query the properties on a legal entity using an instrument identifier.

In [34]:
# get Legal Entity properties via instrument ISIN

index_no = 0
df_inst_LE_rela = pd.DataFrame()

for i in df_inst_unique.index:
    
    inst_LE_rela2 = instruments_api.get_instruments(identifier_type='LusidInstrumentId',
                                                 request_body=[resolver(inst_scope, 'Isin',df_inst_unique['ISIN'][i]),
                                                              ],
                                                 scope=inst_scope,
                                                 relationship_definition_ids=[f"{id_scope}/issuer"],
                                                 property_keys=[f"LegalEntity/{id_scope}/LEI",
                                                                f"LegalEntity/{dei_scopeA}/TotalDEIScore",
                                                                f"LegalEntity/{dei_scopeB}/RaceEthnicityScore"
                                                               ],
                                                )
    df_inst_LE_rela2 = lusid_response_to_data_frame(inst_LE_rela2,rename_properties=True,column_name_mapping=column_rename_mapping)
    df_inst_LE_rela2 = df_inst_LE_rela2.transpose()
    luid = resolver(inst_scope, 'Isin', df_inst_unique['ISIN'][i])
    df_inst_LE_rela2['index_no'] = [f'{index_no}']
    index_no+=1
    
    df_inst_LE_rela2.rename(columns={f'values.{luid}.scope':'scope',
                                     f'values.{luid}.lusid_instrument_id':'lusid_instrument_id',
                                     f'values.{luid}.identifiers.Isin':'instrumentIsin',
                                     f'values.{luid}.relationships.0.related_entity.display_name':'instrumentDisplayName',
                                     f'values.{luid}.relationships.0.related_entity.properties.LegalEntity/deiDataSourceB/RaceEthnicityScore.value.metric_value.value':'raceEthnicityScore',
                                     f'values.{luid}.relationships.0.related_entity.properties.LegalEntity/deiDataSourceA/TotalDEIScore.value.metric_value.value':'totalDEIScore'                               
                                    },inplace=True)
    df_inst_LE_rela = pd.concat([df_inst_LE_rela,df_inst_LE_rela2])

    df_inst_LE_rela.set_index(df_inst_LE_rela['index_no'],inplace=True)
    

df_inst_LE_rela[['instrumentDisplayName',
                 'instrumentIsin',
                 'totalDEIScore',
                 'raceEthnicityScore',
                ]].dropna()


## Filter an instrument universe based on LE DEI data

Now that we have instruments linked to legal entities, we can create a universe of instruments based on the diversity data attached to their corresponding legal entities.

In [35]:
# listing all legal entities in scope that meet DEI criteria

LEs=legal_entities_api.list_legal_entities(id_type_scope=id_scope,
                                             id_type_code="LEI",
                                             property_keys=[f"LegalEntity/{dei_scopeA}/TotalDEIScore",
                                                           f"LegalEntity/{dei_scopeB}/RaceEthnicityScore"],
                                             
                                             # filter by best DEI scores
                                             filter=f"properties[LegalEntity/{dei_scopeA}/TotalDEIScore] gt 45 and properties[LegalEntity/{dei_scopeB}/RaceEthnicityScore] gt 17",
                                             limit=110)

df_LEs = lusid_response_to_data_frame(LEs, column_name_mapping=column_rename_mapping)
df_filtered_LEs = df_LEs[["displayName",
                          "lusidLegalEntityId",
                          "customLegalEntityIdentifier",
                          "legalEntityIdentifier",
                          "totalDEIScore",
                          "raceEthnicityScore"
                         ]]
df_filtered_LEs

In [36]:
# getting instruments related to filtered legal entity list

relationships = []
LEIs = []

for i in df_filtered_LEs.index:
    inst_relationship = legal_entities_api.get_legal_entity_relationships(id_type_scope=id_scope,
                                                  id_type_code="LEI",
                                                  code= df_filtered_LEs["legalEntityIdentifier"][i])
    relationships.append(inst_relationship)
    LEIs.append(df_filtered_LEs["legalEntityIdentifier"][i])

In [37]:
# add the legal entity identifiers to the DataFrame
df_relationships = lusid_response_to_data_frame(relationships, 
                                                rename_properties=True,
                                                column_name_mapping=column_rename_mapping)

df_relationships["legalEntityIdentifier"]=LEIs
df_relationships.head()

In [38]:
# filter to ensure only LE-instrument relationships
df_inst_relationships = df_relationships[df_relationships["values.0.related_entity.entity_type"] == "Instrument"]
df_inst_relationships.rename(columns = {"displayName":"instrumentDisplayName"}, inplace=True)
df_inst_relationships = df_inst_relationships[["instrumentDisplayName",
                                               "instrumentIsin",
                                               "legalEntityIdentifier"]]
df_inst_relationships.head()

Now we can merge DataFrames so you can see the universe of ISINs meeting the DEI score criteria along with the LEI of their legal entity and the Total DEI Score.

In [39]:
merged_df = df_inst_relationships.merge(df_filtered_LEs, left_on="legalEntityIdentifier", right_on="legalEntityIdentifier")
filtered_universe = pd.DataFrame(merged_df[["instrumentDisplayName",
                                            "instrumentIsin",
                                            "legalEntityIdentifier",
                                            "totalDEIScore",
                                            "raceEthnicityScore"]])
filtered_universe